<a href="https://colab.research.google.com/github/blazingbhavneek/scikit-learn_projects/blob/master/Binary_Image_Classifier_by_Bhavneek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sports-Conditional Exercise Image Binary Classifier
## By Bhavneek Singh (IIT Delhi)

This project is part of internship assessment test by i'm beside you. Dataset and task description was given the company itself.

## Initial Assessment
On downloading and looking at the dataset, one can think of a Naive approach of training a CNN, given the `conditioning exercise` images seems to have a lot in common with each other than the `sports` group (i.e sports images are very diverse, a good chance that model wont be able to detect common features)

## Naive Approach: Basic CNN


In [2]:
# Importing all necessary libraries
import tensorflow as tf
from tensorflow.keras.utils import plot_model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Activation, Conv2D, Dense, Flatten,
                                     MaxPool2D)

from tensorflow.keras.optimizers import Adam, SGD

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import os

import random

import zipfile

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import pandas as pd

In [3]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/data.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [6]:
# Walk through data directory and list number of files
for dirpath, dirnames, filenames in os.walk("/tmp/data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 1 images in '/tmp/data'.
There are 2 directories and 1 images in '/tmp/data/test'.
There are 0 directories and 337 images in '/tmp/data/test/conditioning exercise'.
There are 0 directories and 690 images in '/tmp/data/test/sports'.
There are 2 directories and 1 images in '/tmp/data/train'.
There are 0 directories and 1345 images in '/tmp/data/train/conditioning exercise'.
There are 0 directories and 2760 images in '/tmp/data/train/sports'.


In [9]:
# Get the class names
import pathlib
import numpy as np
data_dir = pathlib.Path("/tmp/data/test/")
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
class_names = class_names[1:]
class_names

array(['conditioning exercise', 'sports'], dtype='<U21')

## Pre-Processing

In [10]:
# Set the seed
tf.random.set_seed(42)

# scaling
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "/tmp/data/train/"
test_dir = "/tmp/data/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, 
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)


Found 4105 images belonging to 2 classes.
Found 1027 images belonging to 2 classes.


# Initial Model

In [13]:
# A CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model = tf.keras.models.Sequential([
  Conv2D(filters=10,
					kernel_size=3,
					activation="relu", 
					input_shape=(224, 224, 3)
	),

  Conv2D(10, 3, activation="relu"),
  MaxPool2D(pool_size=2, 
            padding="valid"), 
  Conv2D(10, 3, activation="relu"),
  Conv2D(10, 3, activation="relu"),
  MaxPool2D(2),
  Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") 
])

In [14]:
# Compile the model
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])


In [15]:
# Fit the model
history = model.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))


Epoch 1/5
129/129 [==============================] - 469s 4s/step - loss: 0.5952 - accuracy: 0.6967 - val_loss: 0.5247 - val_accuracy: 0.7390
Epoch 2/5
129/129 [==============================] - 458s 4s/step - loss: 0.5183 - accuracy: 0.7398 - val_loss: 0.5043 - val_accuracy: 0.7371
Epoch 3/5
129/129 [==============================] - 463s 4s/step - loss: 0.4412 - accuracy: 0.7946 - val_loss: 0.4277 - val_accuracy: 0.8208
Epoch 4/5
129/129 [==============================] - 469s 4s/step - loss: 0.3236 - accuracy: 0.8672 - val_loss: 0.4174 - val_accuracy: 0.8238
Epoch 5/5
129/129 [==============================] - 467s 4s/step - loss: 0.2395 - accuracy: 0.9050 - val_loss: 0.4624 - val_accuracy: 0.8218
